In [1]:
import tensorflow as tf

In [2]:
t = tf.convert_to_tensor([[1,2,3],[4,5,6]])

In [3]:
type(t)

tensorflow.python.framework.ops.EagerTensor

In [4]:
t

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [5]:
t.numpy()

array([[1, 2, 3],
       [4, 5, 6]])

In [6]:
t_float = tf.cast(t, dtype=tf.float32)

In [7]:
t_float

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

#### 텐서플로 데이터셋

In [8]:
ds = tf.data.Dataset.from_tensor_slices([5,2,7,9,3])

In [9]:
ds

<TensorSliceDataset shapes: (), types: tf.int32>

In [10]:
for ex in ds:
    print(ex)

tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


In [11]:
#지도학습 : 특성행렬, 레이블벡터

In [12]:
#두 개의 데이터셋을 하나의 데이터셋으로 연결하기 : X에 y 붙이기
X = tf.data.Dataset.from_tensor_slices([[1,2,3],[4,5,6],[9,9,9]])

In [13]:
y = tf.data.Dataset.from_tensor_slices([0,1,0])

In [14]:
my_ds = tf.data.Dataset.zip((X,y))

In [15]:
my_ds

<ZipDataset shapes: ((3,), ()), types: (tf.int32, tf.int32)>

In [16]:
for ex in my_ds:
    print(ex)

(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3])>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)
(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([4, 5, 6])>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)
(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([9, 9, 9])>, <tf.Tensor: shape=(), dtype=int32, numpy=0>)


In [17]:
for ex in my_ds:
    print(ex[0].numpy(), ex[1].numpy())

[1 2 3] 0
[4 5 6] 1
[9 9 9] 0


#### 텐서플로 데이터셋 API

In [18]:
import numpy as np
import tensorflow as tf

In [19]:
ds_x = tf.data.Dataset.from_tensor_slices([[1,2],[3,4],[5,6],[7,8],[9,10.0]])

In [20]:
ds_y = tf.data.Dataset.from_tensor_slices([0,1,2,3,4])

In [21]:
ds_joint = tf.data.Dataset.zip((ds_x, ds_y)) #연결

In [22]:
def display(d):
    for ex in d:
        print(ex[0].numpy(),ex[1].numpy())

In [23]:
display(ds_joint)

[1. 2.] 0
[3. 4.] 1
[5. 6.] 2
[7. 8.] 3
[ 9. 10.] 4


In [24]:
ds_map = ds_joint.map(lambda x, y: (x/10.0, y))

In [25]:
display(ds_map)

[0.1 0.2] 0
[0.3 0.4] 1
[0.5 0.6] 2
[0.7 0.8] 3
[0.9 1. ] 4


In [26]:
ds = ds_map.shuffle(buffer_size = 100)

In [27]:
display(ds)

[0.1 0.2] 0
[0.7 0.8] 3
[0.5 0.6] 2
[0.3 0.4] 1
[0.9 1. ] 4


In [28]:
ds_batch = ds.batch(batch_size = 2)

In [29]:
display(ds_batch) # [[0.7 0.8] [0.9 1. ]] [3 4] 첫번째 batxh, [[0.5 0.6] [0.3 0.4]] [2 1] 두번째 batch, 마지막은 써도 괜찮 안써도 괜찮

[[0.7 0.8]
 [0.1 0.2]] [3 0]
[[0.3 0.4]
 [0.9 1. ]] [1 4]
[[0.5 0.6]] [2]


In [30]:
import tensorflow_datasets as tfds

In [31]:
iris = tfds.load('iris')

In [32]:
type(iris)

dict

In [33]:
iris # features: (4,), label: ()은 딕셔너리

{'train': <PrefetchDataset shapes: {features: (4,), label: ()}, types: {features: tf.float32, label: tf.int64}>}

In [34]:
ds = iris['train'] #150개를 train이라는 value로 줬기 때문에 train으로 씀

In [35]:
ds = ds.shuffle(buffer_size=150, reshuffle_each_iteration=False)

In [36]:
ds_train = ds.take(100)
ds_test = ds.skip(100)

In [37]:
ds_train = ds_train.map(lambda x: (x['features'], x['label'])) # x라는 딕셔너리에서 feature과 label을 뽑아냄
ds_test = ds_test.map(lambda x: (x['features'], x['label'])) # x라는 딕셔너리에서 feature과 label을 뽑아냄

In [38]:
iris_mlp = tf.keras.Sequential([ 
    tf.keras.layers.Dense(16, input_shape=(4,), activation='sigmoid'), 
    tf.keras.layers.Dense(3, activation='softmax')])
iris_mlp.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [39]:
TRAIN_SIZE = 100
NUM_EPOCH = 50
BATCH_SIZE = 2
STEPS_PER_EPOCH = TRAIN_SIZE / BATCH_SIZE

In [40]:
ds_train = ds_train.shuffle(TRAIN_SIZE).batch(BATCH_SIZE)

In [41]:
history = iris_mlp.fit(ds_train, epochs=NUM_EPOCH, steps_per_epoch=STEPS_PER_EPOCH)

Epoch 1/50
50/50 [==============================] - 0s 1ms/step - loss: 1.3192 - accuracy: 0.3800 
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 1.1640 - accuracy: 0.3800
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 1.1113 - accuracy: 0.3800
Epoch 4/50
50/50 [==============================] - 0s 997us/step - loss: 1.0781 - accuracy: 0.3900
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 1.0466 - accuracy: 0.4700
Epoch 6/50
50/50 [==============================] - 0s 855us/step - loss: 1.0225 - accuracy: 0.4400
Epoch 7/50
50/50 [==============================] - 0s 997us/step - loss: 0.9958 - accuracy: 0.5600
Epoch 8/50
50/50 [==============================] - 0s 957us/step - loss: 0.9639 - accuracy: 0.7600
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.9351 - accuracy: 0.8000
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.8969 - accuracy: 0.9300
Epoch 11/5

In [42]:
iris_mlp.evaluate(ds_test.batch(50))

1/1 [==============================] - 0s 141ms/step - loss: 0.3330 - accuracy: 0.9400


[0.3329547047615051, 0.9399999976158142]